In [ ]:
import json
import requests

In [ ]:
# 용존산소, 인산염인을 지닌 api
key1 = 'xxx' 

# 아질산염질소, 용존무기질소를 지닌 api
key2 = 'xxx' #비공개

# api 날짜 범위
start_date_2019 = '20190101'
end_date_2019 = '20191231'
start_date_2020 = '20200101'
end_date_2020 = '20201231'
start_date_2021 = '20210101'
end_date_2021 = '20211231'

# api url
url_2019a = f"https://www.nifs.go.kr/OpenAPI_json?id=sooList&key={key1}&sdate={start_date_2019}&edate={end_date_2019}"
url_2020a = f"https://www.nifs.go.kr/OpenAPI_json?id=sooList&key={key1}&sdate={start_date_2020}&edate={end_date_2020}"
url_2021a = f"https://www.nifs.go.kr/OpenAPI_json?id=sooList&key={key1}&sdate={start_date_2021}&edate={end_date_2021}"

url_2019b = f"https://www.nifs.go.kr/OpenAPI_json?id=femoSeaList&key={key2}&sdate={start_date_2019}&edate={end_date_2019}"
url_2020b = f"https://www.nifs.go.kr/OpenAPI_json?id=femoSeaList&key={key2}&sdate={start_date_2020}&edate={end_date_2020}"
url_2021b = f"https://www.nifs.go.kr/OpenAPI_json?id=femoSeaList&key={key2}&sdate={start_date_2021}&edate={end_date_2021}"

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

데이터 받아오기

In [ ]:
temp =requests.get(url_2019a)
temp.encoding='euc-kr'
temp_2019a = temp.json()

temp =requests.get(url_2020a)
temp.encoding='euc-kr'
temp_2020a = temp.json()

temp =requests.get(url_2021a)
temp.encoding='euc-kr'
temp_2021a = temp.json()

### 1번 api 정보

데이터 확인

In [ ]:
temp_2021a

In [ ]:
temp_2021a['body'].keys()

In [ ]:
temp_2021a['body']['item']

원본 데이터 : df_20XXa, df_20XX_b  
복사 데이터 : copy_df_20XXa, copy_df_20XXb

In [ ]:
df_2019a = pd.DataFrame(temp_2019a['body']['item'])
df_2020a = pd.DataFrame(temp_2020a['body']['item'])
df_2021a = pd.DataFrame(temp_2021a['body']['item'])

In [ ]:
copy_df_2021a = pd.DataFrame(df_2021a)
copy_df_2021a.head(3)

사용하지 않을 column 삭제

In [ ]:
copy_df_2021a.drop(labels=['nut_no3_n','sln_cde', 'sta_cde','wtr_tmp', 'wtr_dep', 'qc_wtr', 
                      'sal', 'qc_sal','qc_dox','nut_no2_n', 'nut_no3_n','nut_sio2_si',
                      'nut_ph','wtr_trn','atm','res_vsl_nm'],
             axis=1, inplace=True)

copy_df_2021a.head()

column명 통합 및 순서 정리

In [ ]:
copy_df_2021a.columns = ['경도','해역','용존산소','관측일시','인산염인','위도']
copy_df_2021a.head()

In [ ]:
copy_df_2021a['위도/경도'] = copy_df_2021a[['위도','경도']].apply(
    lambda row : ' / '.join(row),
    axis = 1
)

In [ ]:
copy_df_2021a.drop(labels=['위도','경도'],
              axis=1, inplace=True)

In [ ]:
copy_df_2021a.head()

In [ ]:
copy_df_2021a = copy_df_2021a.sort_values(by=['관측일시'],
                                ascending=True)
copy_df_2021a.head()

In [ ]:
copy_df_2021a.info()

In [ ]:
copy_df_2021a.set_index('위도/경도', inplace = True)
copy_df_2021a.head()

In [ ]:
copy_df_2021a = copy_df_2021a[['해역','관측일시','용존산소','인산염인']]
copy_df_2021a.head()

### !!!결측치 인코딩 작업 필요!!!
---

### 2번 api 정보

데이터 불러오기

In [ ]:
temp =requests.get(url_2021b)
temp.encoding='euc-kr'
temp_2021b = temp.json()

데이터 확인

In [ ]:
temp_2021b

In [ ]:
temp_2021b['body']['item']

원본 / 카피데이터 분리

In [ ]:
df_2021b = pd.DataFrame(temp_2021b['body']['item'])
copy_df_2021b = pd.DataFrame(df_2021b)
copy_df_2021b.head()

In [ ]:
copy_df_2021b.info()

사용하지 않을 column 삭제

In [ ]:
copy_df_2021b.drop(labels=['CHL_B', 'SIL_S', 'SAL_B', 'NH4_N_B', 'M', 'DIP_S', 'DO_B','CHL_S', 'TP_B',
                     'WEATHER', 'NH4_N_S','SAL_S', 'DIP_B', 'TN_S', 'DEPTH', 'DO_S', 'PH_B',
                     'KIND', 'COD_B',  'SS_S', 'NO3_N_S', 'COD_S', 'PH_S', 'TN_B', 'SIL_B',
                     'LOCATION_POINT', 'SS_B','TP_S','NO3_N_B','TEMP_S','TEMP_B'], 
              inplace=True, axis =1)

copy_df_2021b.head()

column통합, 정리

In [ ]:
print(copy_df_2021b['TIME_H'].sort_values(ascending = False).unique())
print(copy_df_2021b['TIME_I'].sort_values(ascending = False).unique())

In [ ]:
copy_df_2021b['TIME_H'].replace(['0','1','2','3','4','5','6','7','8','9'], 
                           ['00','01','02','03','04','05','06','07','08','09'],  
                           inplace= True)

print(copy_df_2021b['TIME_H'].unique())

In [ ]:
copy_df_2021b['TIME_I'].replace(['0','1','2','3','4','5','6','7','8','9'], 
                           ['00','01','02','03','04','05','06','07','08','09'], 
                           inplace=True)
print(copy_df_2021b['TIME_I'].unique())

In [ ]:
print(copy_df_2021b['DATE_M'].unique())
print(copy_df_2021b['DATE_D'].unique())

In [ ]:
copy_df_2021b['DATE_M'].replace(['0','1','2','3','4','5','6','7','8','9'],
                           ['00','01','02','03','04','05','06','07','08','09'],
                           inplace=True)

copy_df_2021b['DATE_D'].replace(['0','1','2','3','4','5','6','7','8','9'],
                           ['00','01','02','03','04','05','06','07','08','09'],
                           inplace=True)


print(copy_df_2021b['DATE_M'].unique())
print(copy_df_2021b['DATE_D'].unique())

In [ ]:
copy_df_2021b['년월일'] = copy_df_2021b[['DATE_Y','DATE_M','DATE_D']].apply(
    lambda row : ':'.join(row),
    axis =1
)

copy_df_2021b['시간'] = copy_df_2021b[['TIME_H','TIME_I']].apply(
    lambda row : ':'.join(row),
    axis = 1
)

copy_df_2021b['관측일시'] = copy_df_2021b[['년월일','시간']].apply(
    lambda row : ' '.join(row),
    axis = 1
    
)

copy_df_2021b.drop(labels=['DATE_Y','DATE_M','DATE_D','TIME_H','TIME_I', '년월일', '시간'],
              axis = 1, inplace = True)

In [ ]:
copy_df_2021b['위도/경도'] = copy_df_2021b[['LATITUDE','LONGITUDE']].apply(
    lambda row : ' / '.join(row),
    axis = 1
)

copy_df_2021b.drop(labels=['LATITUDE', 'LONGITUDE'], 
              axis =1, inplace=True)

In [ ]:
copy_df_2019b = copy_df_2021b.sort_values(by=['관측일시'], ascending=True)
copy_df_2021b.head()

In [ ]:
copy_df_2021b.set_index('위도/경도', inplace=True)
copy_df_2021b.columns = ['아질산염질소-저층', '아질산염질소-표층', '용존무기질소-저층', '용존무기질소-표층', '어장명', '관측일시']
copy_df_2021b = copy_df_2021b[['어장명','관측일시','아질산염질소-저층', '아질산염질소-표층','용존무기질소-저층', '용존무기질소-표층']]
copy_df_2021b.head()

촤종확인

In [ ]:
copy_df_2021a.info()

In [ ]:
copy_df_2021a.head(3)

In [ ]:
copy_df_2021b.info()

In [ ]:
copy_df_2021b.head(3)

자동화코드

In [ ]:
import json
import requests
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

start = '20210101'
end= '20211231'
path = r'C:\Users\smhrd\Desktop\project\data'

In [ ]:
def make_a(start,end) :
    
    key = 'qPwOeIrU-2208-01rnrwls-0468' 
    url = f"https://www.nifs.go.kr/OpenAPI_json?id=sooList&key={key}&sdate={start}&edate={end}"
    
    temp = requests.get(url)
    temp.encoding = 'euc-kr'
    temp_a = temp.json()
    
    df_a = pd.DataFrame(temp_a['body']['item'])
    
    df_a.drop(labels=['nut_no3_n','sln_cde', 'sta_cde','wtr_tmp', 'wtr_dep', 'qc_wtr', 
                      'sal', 'qc_sal','qc_dox','nut_no2_n', 'nut_no3_n','nut_sio2_si',
                      'nut_ph','wtr_trn','atm','res_vsl_nm'],
             axis=1, inplace=True)
    df_a.columns = ['경도','해역','용존산소','관측일시','인산염인','위도']
    
    df_a['위도/경도'] = df_a[['위도','경도']].apply(lambda row : ' / '.join(row), axis = 1)
    df_a.drop(labels=['위도','경도'], axis=1, inplace=True)
    # df_a.set_index('위도/경도', inplace = True)
    
    df_a = df_a.sort_values(by=['관측일시'], ascending=True)
    # df_a = df_a[['해역','관측일시','용존산소','인산염인']]
    
    # df_a.to_csv(path+f'/{start[:4]}(a).csv', index_label=False)
    return df_a

In [ ]:
def make_b(start,end) :
    
    key = 'qPwOeIrU-2208-01rnrwls-0467' 
    url = f'https://www.nifs.go.kr/OpenAPI_json?id=femoSeaList&key={key}&sdate={start}&edate={end}'
    
    temp = requests.get(url)
    temp.encoding = 'euc-kr'
    temp_b = temp.json()
    
    df_b = pd.DataFrame(temp_b['body']['item'])
    df_b.drop(labels=['CHL_B', 'SIL_S', 'SAL_B', 'NH4_N_B', 'M', 'DIP_S', 'DO_B','CHL_S', 'TP_B',
                      'WEATHER', 'NH4_N_S','SAL_S', 'DIP_B', 'TN_S', 'DEPTH', 'DO_S', 'PH_B',
                      'KIND', 'COD_B',  'SS_S', 'NO3_N_S', 'COD_S', 'PH_S', 'TN_B', 'SIL_B',
                      'LOCATION_POINT', 'SS_B','TP_S','NO3_N_B','TEMP_S','TEMP_B'], 
               inplace=True, axis =1)
    
    df_b['TIME_H'].replace(['0','1','2','3','4','5','6','7','8','9'], 
                            ['00','01','02','03','04','05','06','07','08','09'],  
                            inplace= True)
    df_b['TIME_I'].replace(['0','1','2','3','4','5','6','7','8','9'], 
                            ['00','01','02','03','04','05','06','07','08','09'], 
                            inplace=True)
    df_b['DATE_M'].replace(['0','1','2','3','4','5','6','7','8','9'],
                            ['00','01','02','03','04','05','06','07','08','09'],
                            inplace=True)
    df_b['DATE_D'].replace(['0','1','2','3','4','5','6','7','8','9'],
                            ['00','01','02','03','04','05','06','07','08','09'],
                            inplace=True)
    
    df_b['년월일'] = df_b[['DATE_Y','DATE_M','DATE_D']].apply(
     lambda row : ':'.join(row),axis =1)
    df_b['시간'] = df_b[['TIME_H','TIME_I']].apply(
     lambda row : ':'.join(row),axis = 1)
    df_b['관측일시'] = df_b[['년월일','시간']].apply(
     lambda row : ' '.join(row),
     axis = 1)
    df_b.drop(labels=['DATE_Y','DATE_M','DATE_D','TIME_H','TIME_I', '년월일', '시간'],
               axis = 1, inplace = True)
    
    df_b['위도/경도'] = df_b[['LATITUDE','LONGITUDE']].apply(lambda row : ' / '.join(row),axis = 1)
    df_b.drop(labels=['LATITUDE', 'LONGITUDE'], axis =1, inplace=True)
    df_b.set_index('위도/경도', inplace=True)
    
    df_b.columns = ['아질산염질소-저층', '아질산염질소-표층', '용존무기질소-저층', '용존무기질소-표층', '어장명', '관측일시']
    df_b = df_b[['어장명','관측일시','아질산염질소-저층', '아질산염질소-표층','용존무기질소-저층', '용존무기질소-표층']]
    
    df_b.to_csv(path+f'/{start[:4]}(b).csv', index_label=False)
    
    return df_b
 

In [ ]:
df_a = make_a(start,end)

In [ ]:
df_a.head()

김 데이터 현황 확인

In [ ]:
df_b = make_b(start,end)

In [ ]:
df_b['FISHERY'].unique()

In [ ]:
df_ndg = df_b[df_b['FISHERY']=='낙동강하구']
df_busan = df_b[df_b['FISHERY']=='부산']

In [ ]:
df_ndg[df_ndg['KIND']=='김'].value_counts()

In [ ]:
df_busan[df_busan['KIND']=='김'].value_counts()

In [ ]:
np.unique(df_busan['KIND'], return_counts=True)

In [ ]:
df_busan['KIND'].value_counts()

In [ ]:
df_b[df_b['KIND']=='김']['FISHERY'].value_counts()